## GET COMPANY LIST

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re  # Import regex for cleaning numbers

# Set up Selenium WebDriver (Update the path to your ChromeDriver)
chrome_driver_path = "C:\\chromedriver-win64\\chromedriver.exe"  # Update this if necessary
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.cse.lk/pages/market-capitalization/market-capitalization.html"
driver.get(url)

# Initialize empty list to store company data
company_data = []
page_number = 1  # Track the page number

# Function to extract data from the current page
def extract_data():
    global page_number
    print(f"\n📌 Scraping Page {page_number}...")

    # Wait for the table to load
    wait = WebDriverWait(driver, 10)
    table = wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

    # Extract table headers to determine correct column positions
    headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, "th")]
    
    # Identify column indexes
    try:
        name_index = headers.index("Company Name")
        market_cap_index = headers.index("Market Capitalization")
    except ValueError:
        print("❌ Error: Could not find required column headers.")
        return

    # Extract table rows (excluding the header row)
    rows = table.find_elements(By.TAG_NAME, "tr")[1:]  
    companies_collected = 0  # Track how many companies were collected

    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) > market_cap_index:  # Ensure enough columns exist
            company_name = cells[name_index].text.strip()  # Extract company name
            market_cap = cells[market_cap_index].text.strip()  # Extract market capitalization
            
            # Clean and extract numeric value using regex
            market_cap_cleaned = re.sub(r"[^\d.]", "", market_cap)  # Remove non-numeric characters
            
            try:
                market_cap_value = float(market_cap_cleaned) if market_cap_cleaned else 0
                company_data.append([company_name, market_cap_value])
                companies_collected += 1
            except ValueError:
                print(f"⚠️ Skipping invalid market cap for {company_name}: {market_cap}")

    print(f"✅ Page {page_number} done! {companies_collected} companies extracted.")
    page_number += 1  # Increment page number

# Function to navigate to the next page and extract data
def scrape_all_pages():
    while True:
        try:
            # Extract data from the current page
            extract_data()
            
            # Check if the "Next" button exists and is clickable
            next_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Next')]"))
            )
            
            # Check if the button is **disabled**
            if "disabled" in next_button.get_attribute("class").lower():
                print("\n🚀 No more pages to scrape. Stopping now!")
                break
            
            # Click the "Next" button
            next_button.click()
            time.sleep(3)  # Wait for the next page to load
        except Exception as e:
            print("\n🚀 No more pages or error navigating to the next page. Scraping completed!")
            break

# Scrape all pages of company data
scrape_all_pages()

# Close the browser
driver.quit()

# Create DataFrame and sort by Market Capitalization (Descending)
df = pd.DataFrame(company_data, columns=['Company Name', 'Market Capitalization'])
df_sorted = df.sort_values(by='Market Capitalization', ascending=False)

# Print top results before saving
print("\n🔝 Top 5 Companies by Market Capitalization:")
print(df_sorted.head())

# Save sorted companies to a CSV file
df_sorted.to_csv("top_cse_companies_by_market_cap.csv", index=False)

print("\n✅ Top companies sorted by Market Capitalization successfully saved to top_cse_companies_by_market_cap.csv")
print(f"📊 Total Companies Collected: {len(df_sorted)}")



📌 Scraping Page 1...
✅ Page 1 done! 25 companies extracted.

📌 Scraping Page 2...
✅ Page 2 done! 25 companies extracted.

📌 Scraping Page 3...
✅ Page 3 done! 25 companies extracted.

📌 Scraping Page 4...
✅ Page 4 done! 25 companies extracted.

📌 Scraping Page 5...
✅ Page 5 done! 25 companies extracted.

📌 Scraping Page 6...
✅ Page 6 done! 25 companies extracted.

📌 Scraping Page 7...
✅ Page 7 done! 25 companies extracted.

📌 Scraping Page 8...
✅ Page 8 done! 25 companies extracted.

📌 Scraping Page 9...
✅ Page 9 done! 25 companies extracted.

📌 Scraping Page 10...
✅ Page 10 done! 25 companies extracted.

📌 Scraping Page 11...
✅ Page 11 done! 25 companies extracted.

📌 Scraping Page 12...
✅ Page 12 done! 25 companies extracted.

🚀 No more pages to scrape. Stopping now!

🔝 Top 5 Companies by Market Capitalization:
                      Company Name  Market Capitalization
144       JOHN KEELLS HOLDINGS PLC           3.295806e+11
157           L O L C HOLDINGS PLC           2.572020e+11
7

## SEARCH COMPANY URLS USING ZENROW API FOR CAPTCHA BYPASS

In [2]:
# !pip install selenium zenrows pandas

In [3]:
from zenrows import ZenRowsClient
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# ✅ Load company names from CSV
df = pd.read_csv("top_cse_companies_by_market_cap.csv")  # Ensure CSV has 'Company Name' column
company_names = df["Company Name"].tolist()

# ✅ Set up Selenium WebDriver
chrome_driver_path = "C:\\chromedriver-win64\\chromedriver.exe"  # Update this path if needed
service = Service(chrome_driver_path)

# Chrome options (Avoid detection)
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")  # Helps avoid detection
options.add_argument("--start-maximized")  # Opens the browser in full screen
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
)  # Fake real browser user-agent

# Initialize WebDriver
driver = webdriver.Chrome(service=service, options=options)

# ✅ Initialize ZenRows API
ZENROWS_API_KEY = "ac4a623d6652bde68c4e021e607f3e0d814e0516"
client = ZenRowsClient(ZENROWS_API_KEY)

# ✅ Store results
results = []

def search_google(company):
    """Perform Google search and extract the first result URL."""
    query = f"{company} Board of Directors"
    driver.get("https://www.google.com")

    # Locate search box and enter query
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(query)
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(3)  # Wait for results to load

    try:
        # Get the first search result link
        first_result = driver.find_element(By.CSS_SELECTOR, "h3")
        first_result_parent = first_result.find_element(By.XPATH, "./ancestor::a")  # Get parent <a> tag
        first_result_url = first_result_parent.get_attribute("href")
    except Exception:
        first_result_url = "No relevant results found"

    return first_result_url

def scrape_with_zenrows(url):
    """Scrape the given URL using ZenRows to bypass bot detection."""
    if url == "No relevant results found":
        return "No content found"

    try:
        response = client.get(url)
        return response.text  # Returns full HTML content
    except Exception as e:
        return f"Error scraping: {str(e)}"

# ✅ Loop through companies
for company in company_names:
    print(f"Searching for: {company}...")
    board_link = search_google(company)
    print(f"Company: {company} | Board Link: {board_link}")

    # Scrape the page content using ZenRows
    board_page_content = scrape_with_zenrows(board_link)
    
    results.append({
        "Company Name": company,
        "Board_Link": board_link,
        "Board_Page_Content": board_page_content[:500]  # Storing first 500 characters to avoid large CSV
    })
    
    time.sleep(5)  # Pause between searches to avoid Google blocking

# ✅ Save to CSV
df_results = pd.DataFrame(results)
df_results.to_csv("board_of_directors_combined.csv", index=False)

# ✅ Close browser
driver.quit()

print("\n✅ Search completed. Results saved in 'board_of_directors_combined.csv'.")


Searching for: JOHN KEELLS HOLDINGS PLC...
Company: JOHN KEELLS HOLDINGS PLC | Board Link: https://www.keells.com/leadership/
Searching for: L O L C HOLDINGS PLC...
Company: L O L C HOLDINGS PLC | Board Link: https://www.lolc.com/directors
Searching for: CEYLON TOBACCO COMPANY PLC...
Company: CEYLON TOBACCO COMPANY PLC | Board Link: https://www.ceylontobaccocompany.com/who-we-are/corporate-governance/our-leadership
Searching for: COMMERCIAL BANK OF CEYLON PLC...
Company: COMMERCIAL BANK OF CEYLON PLC | Board Link: https://www.combank.lk/about-us
Searching for: LOLC FINANCE PLC...
Company: LOLC FINANCE PLC | Board Link: https://www.lolcfinance.com/about-us/board-of-directors/
Searching for: DISTILLERIES COMPANY OF SRI LANKA PLC...
Company: DISTILLERIES COMPANY OF SRI LANKA PLC | Board Link: https://www.marketscreener.com/quote/stock/DISTILLERIES-COMPANY-OF-S-6495013/company/
Searching for: MELSTACORP PLC...
Company: MELSTACORP PLC | Board Link: https://melstacorp.com/group/directors
Sea

## SEPERATE THE WEB LINKS AND PDF LINKS

In [4]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("board_of_directors_combined.csv")

# Ensure 'Board_Link' column exists and drop NaN values
df = df.dropna(subset=["Board_Link"])

# Filter PDF links
df_pdfs = df[df["Board_Link"].str.endswith(".pdf", na=False)]
pdf_count = len(df_pdfs)  # Count PDF links

# Filter non-PDF links
df_web = df[~df["Board_Link"].str.endswith(".pdf", na=False)]
web_count = len(df_web)  # Count web links

# Save filtered results
df_pdfs.to_csv("pdf_links.csv", index=False)  # Save PDFs
df_web.to_csv("web_links.csv", index=False)  # Save non-PDFs

# Print counts
print(f"✅ PDF links saved in 'pdf_links.csv' (Total: {pdf_count})")
print(f"✅ Non-PDF links saved in 'web_links.csv' (Total: {web_count})")


✅ PDF links saved in 'pdf_links.csv' (Total: 26)
✅ Non-PDF links saved in 'web_links.csv' (Total: 274)


### Remove wsj URLs

In [5]:
import pandas as pd

# Load the existing results
df_results = pd.read_csv("web_links.csv")

# Filter rows where Board_Link contains 'www.wsj.com'
df_wsj = df_results[df_results["Board_Link"].str.contains("www.wsj.com", na=False)]
wsj_count = len(df_wsj)  # Count of WSJ links

# Filter out rows that do NOT contain 'www.wsj.com'
df_filtered = df_results[~df_results["Board_Link"].str.contains("www.wsj.com", na=False)]
filtered_count = len(df_filtered)  # Count of remaining links

# Save the filtered results separately
df_wsj.to_csv("wsj.csv", index=False)  # Contains only 'www.wsj.com' links
df_filtered.to_csv("company_urls_filtered.csv", index=False)  # Contains other results

# Print counts
print(f"\n✅ Filtering completed. Saved WSJ links separately in 'wsj.csv' (Total: {wsj_count}).")
print(f"✅ Remaining results saved in 'company_urls_filtered.csv' (Total: {filtered_count}).")



✅ Filtering completed. Saved WSJ links separately in 'wsj.csv' (Total: 63).
✅ Remaining results saved in 'company_urls_filtered.csv' (Total: 211).


## EXTRACT THE DIRECORS NAMES USING WEB LINKS

In [6]:
# !python -m spacy download en_core_web_lg


In [7]:
import warnings
from urllib3.exceptions import InsecureRequestWarning  # Import the warning class
import requests
from bs4 import BeautifulSoup
import spacy
import time
import pandas as pd

# Suppress SSL warnings
warnings.filterwarnings("ignore", category=InsecureRequestWarning)

# Load spaCy's NLP model
nlp = spacy.load("en_core_web_lg")

# Load CSV file containing URLs (Ensure it has 'Company Name' and 'Board_Link' columns)
df = pd.read_csv("company_urls_filtered.csv")  # Replace with your actual CSV file name
companies = df[["Company Name", "Board_Link"]].dropna().to_dict(orient="records")  # Ensure no missing URLs

def clean_name(name):
    """Cleans extracted names by removing unwanted words and ensuring proper format"""
    name = name.strip()
    
    # Remove common incorrect extractions
    exclude_list = ["MarketScreener", "Executive", "Directors", "Chairman", "CEO", "Managing Director", "Non-"
                    "Non-Executive", "Vice Chairman", "CIMA", "Enderby", "Sri Lanka’s", "Largest Conglomerates","Transactions Earnings","Email Newsletters","Governance","Careers","Sri Lanka"]
    
    # Remove titles or unwanted words
    for word in exclude_list:
        name = name.replace(word, "").strip()

    # Ensure name contains at least two words and isn't too short
    if len(name.split()) < 2 or len(name) < 5:
        return None
    
    return name

def extract_directors_nlp(url):
    """Extract names of directors from a webpage using NLP"""
    
    import random

    USER_AGENTS = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
    ]
    
    headers = {"User-Agent": random.choice(USER_AGENTS)}

    try:
        response = requests.get(url, headers=headers, timeout=10, verify=False)  # Disable SSL verification here

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            
            # Extract all visible text
            text = soup.get_text(separator=" ")

            # Apply NLP to find names
            doc = nlp(text)
            persons = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

            # Filter based on keyword proximity
            relevant_names = set()
            keywords = ["Board of Directors", "Chairman", "CEO", "Director", "Leadership", "Management", "Board Member",
                        "Executive Chairman", "Non-Executive Director","Executive Vice Chairman","Managing Director","Executive Director"]

            for keyword in keywords:
                if keyword in text:
                    keyword_pos = text.index(keyword)  # Find keyword position
                    # Extract a smaller context around the keyword (e.g., ±300 characters for better accuracy)
                    context = text[max(0, keyword_pos - 300): keyword_pos + 300]
                    
                    # Extract names from this section only
                    context_doc = nlp(context)
                    relevant_names.update([ent.text for ent in context_doc.ents if ent.label_ == "PERSON"])

            # Clean names
            clean_names = list(filter(None, {clean_name(name) for name in relevant_names}))

            return clean_names, None  # Return names with no error message (None)
        else:
            error_message = f"Failed to fetch page: {response.status_code} | URL: {url}"
            return [], error_message
    except requests.RequestException as e:
        error_message = f"Request failed: {e} | URL: {url}"
        return [], error_message


# Process all URLs with index to show the number
successful_results = []
failed_results = []

for index, company in enumerate(companies, start=1):  # Use enumerate to get the index
    company_name = company["Company Name"]
    url = company["Board_Link"]
    
    print(f"\n🔍 Processing {index}/{len(companies)}: {company_name} ({url})")  # Include the index and total number of companies
    director_names, error_message = extract_directors_nlp(url)
    
    # If extraction is successful, append to successful_results
    if director_names:
        successful_results.append({"Company Name": company_name, "URL": url, "Directors": ", ".join(director_names)})
        print(f"✅ {company_name} - Extracted Board Members:\n" + "\n".join(director_names))
    else:
        # If extraction fails, append to failed_results along with the error message
        failed_results.append({"Company Name": company_name, "URL": url, "Error": error_message})
        print(f"❌ {company_name} - No valid names found.\nError: {error_message}")

    time.sleep(2)  # Avoid getting blocked

# Convert to DataFrames
df_success = pd.DataFrame(successful_results)
df_failed = pd.DataFrame(failed_results)

# # Save to CSV (optional)
# df_success.to_csv("directors_extracted_success.csv", index=False)
# df_failed.to_csv("directors_extracted_failed.csv", index=False)

# Print summary
print("\n✅ Extraction complete!")
print(f"✅ Successful extractions: {len(df_success)}")
print(f"❌ Failed extractions: {len(df_failed)}")





🔍 Processing 1/211: JOHN KEELLS HOLDINGS PLC (https://www.keells.com/leadership/)
✅ JOHN KEELLS HOLDINGS PLC - Extracted Board Members:
Amal Cabraal
Dr Wijayasuriya
Krishan Balendra

🔍 Processing 2/211: L O L C HOLDINGS PLC (https://www.lolc.com/directors)
✅ L O L C HOLDINGS PLC - Extracted Board Members:
Mrs Amarasinghe
K U Amarasinghe
Kalsha Amarasinghe
Kapila Jayawardena
Ravi Fernando
Ravi Fernando 
 Non-

🔍 Processing 3/211: CEYLON TOBACCO COMPANY PLC (https://www.ceylontobaccocompany.com/who-we-are/corporate-governance/our-leadership)
✅ CEYLON TOBACCO COMPANY PLC - Extracted Board Members:
GARY TARRANT
STUART KIDD
FARIYHA SUBHANI
SURESH SHAH

🔍 Processing 4/211: COMMERCIAL BANK OF CEYLON PLC (https://www.combank.lk/about-us)
✅ COMMERCIAL BANK OF CEYLON PLC - Extracted Board Members:
Palitha Kumarasinghe
Thushara Wijewardena
Lakshman Niyangoda
Nimal Fernando
Judy Lee
Sivakumar Selliah
Yohan Perera
Raja Senanayake

🔍 Processing 5/211: LOLC FINANCE PLC (https://www.lolcfinance.com/a

In [8]:
#successful_results

In [9]:
# print("\n📄 Failed Extractions:")
# print(df_failed)


### Retry for Failures

In [10]:
import warnings
from urllib3.exceptions import InsecureRequestWarning
import requests
from bs4 import BeautifulSoup
import spacy
import time
import pandas as pd

# Suppress SSL warnings
warnings.filterwarnings("ignore", category=InsecureRequestWarning)

# Load spaCy NLP model
nlp = spacy.load("en_core_web_lg")


def clean_name(name):
    """Clean extracted names by removing titles and checking name format"""
    name = name.strip()

    exclude_list = [
        "MarketScreener", "Executive", "Directors", "Chairman", "CEO", "Managing Director", "Non-"
        "Non-Executive", "Vice Chairman", "CIMA", "Enderby", "Sri Lanka’s", "Largest Conglomerates",
        "Transactions Earnings", "Email Newsletters", "Governance", "Careers", "Sri Lanka"
    ]

    for word in exclude_list:
        name = name.replace(word, "").strip()

    if len(name.split()) < 2 or len(name) < 5:
        return None

    return name

def extract_directors_nlp(url):
    """Extract director names using spaCy NLP with keyword context window"""
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers, timeout=10, verify=False)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text(separator=" ")

            doc = nlp(text)
            keywords = [
                "Board of Directors", "Chairman", "CEO", "Director", "Leadership", "Management",
                "Board Member", "Executive Chairman", "Non-Executive Director", "Executive Vice Chairman",
                "Managing Director", "Executive Director"
            ]

            relevant_names = set()
            for keyword in keywords:
                if keyword in text:
                    keyword_pos = text.index(keyword)
                    context = text[max(0, keyword_pos - 300): keyword_pos + 300]
                    context_doc = nlp(context)
                    relevant_names.update([ent.text for ent in context_doc.ents if ent.label_ == "PERSON"])

            clean_names = list(filter(None, {clean_name(name) for name in relevant_names}))
            return clean_names, None
        else:
            return [], f"Failed to fetch page: {response.status_code} | URL: {url}"
    except requests.RequestException as e:
        return [], f"Request failed: {e} | URL: {url}"

# Initialize list for new failures only
new_failures = []

# Re-process previously failed entries
successful_results = df_success.to_dict(orient="records")
failed_records = df_failed.to_dict(orient="records")

for idx, failed in enumerate(failed_records, 1):
    total = len(failed_records)
    company_name = failed["Company Name"]
    url = failed["URL"]

    print(f"\n🔍 ({idx}/{total}) Processing: {company_name} ({url})")  # Display progress
    director_names, error_message = extract_directors_nlp(url)

    if director_names:
        successful_results.append({"Company Name": company_name, "URL": url, "Directors": ", ".join(director_names)})
        print(f"✅ {company_name} - Extracted Board Members:\n" + "\n".join(director_names))
    else:
        new_failures.append({
            "Company Name": company_name,
            "URL": url,
            "Error": error_message
        })
        print(f"❌ {company_name} - Still failed.\nError: {error_message}")

    time.sleep(2)  # Be polite to servers

# Create new DataFrame of failed extractions
df_failed = pd.DataFrame(new_failures)
df_success = pd.DataFrame(successful_results)

# Optional: Save the new failed extractions DataFrame to a CSV file
# df_failure.to_csv("new_failed_extractions.csv", index=False)

# Final summary
print("\n🆕 Re-attempt complete.")
print(f"✅ Successful extractions: {len(df_success)}")
print(f"❌ Still failed: {len(df_failed)}")



🔍 (1/26) Processing: DIALOG AXIATA PLC (https://www.dialog.lk/structure-and-management)
✅ DIALOG AXIATA PLC - Extracted Board Members:
DAVID NAI PEK LAU
WILLEM LUCAS

🔍 (2/26) Processing: SAMPATH BANK PLC (https://www.sampath.lk/about-us/leadership)
❌ SAMPATH BANK PLC - Still failed.
Error: None

🔍 (3/26) Processing: BROWNS INVESTMENTS PLC (https://www.brownsinvestments.com/about-us/our-team.html)
❌ BROWNS INVESTMENTS PLC - Still failed.
Error: Failed to fetch page: 403 | URL: https://www.brownsinvestments.com/about-us/our-team.html

🔍 (4/26) Processing: CEYLON COLD STORES PLC (https://www.elephanthouse.lk/corporate/our-company/our-team.html)
❌ CEYLON COLD STORES PLC - Still failed.
Error: None

🔍 (5/26) Processing: DFCC BANK PLC (https://www.dfcc.lk/board-of-directors/)
❌ DFCC BANK PLC - Still failed.
Error: None

🔍 (6/26) Processing: NATIONAL DEVELOPMENT BANK PLC (https://www.ndbbank.com/about/board-of-directors)
❌ NATIONAL DEVELOPMENT BANK PLC - Still failed.
Error: Failed to fetch

In [11]:
new_failures

[{'Company Name': 'SAMPATH BANK PLC',
  'URL': 'https://www.sampath.lk/about-us/leadership',
  'Error': None},
 {'Company Name': 'BROWNS INVESTMENTS PLC',
  'URL': 'https://www.brownsinvestments.com/about-us/our-team.html',
  'Error': 'Failed to fetch page: 403 | URL: https://www.brownsinvestments.com/about-us/our-team.html'},
 {'Company Name': 'CEYLON COLD STORES PLC',
  'URL': 'https://www.elephanthouse.lk/corporate/our-company/our-team.html',
  'Error': None},
 {'Company Name': 'DFCC BANK PLC',
  'URL': 'https://www.dfcc.lk/board-of-directors/',
  'Error': None},
 {'Company Name': 'NATIONAL DEVELOPMENT BANK PLC',
  'URL': 'https://www.ndbbank.com/about/board-of-directors',
  'Error': 'Failed to fetch page: 403 | URL: https://www.ndbbank.com/about/board-of-directors'},
 {'Company Name': 'TEEJAY LANKA PLC',
  'URL': 'https://www.teejay.com/about-teejay/board-of-directors',
  'Error': None},
 {'Company Name': 'BROWN & COMPANY PLC',
  'URL': 'https://www.brownsgroup.lk/about-us/board-of

### Retry again Failures

In [12]:
import time
import warnings
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import spacy

# Load NLP model
nlp = spacy.load("en_core_web_lg")

# Ignore warnings
warnings.filterwarnings("ignore")

successful_results = df_success.to_dict(orient="records")
failed_records = df_failed.to_dict(orient="records")
# # Assume new_failures is provided as a list of failed companies (with 'Company Name' and 'URL' columns)
# df_failure = pd.DataFrame(new_failures).dropna(subset=["URL"])  # Remove empty URLs
# companies = df_failure.to_dict(orient="records")
# successful_results = df_success_updated.to_dict(orient="records")

# Configure Selenium WebDriver (Headless Chrome)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run without opening browser
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

# Function to extract text using Selenium
def fetch_page_content(url):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        return driver.page_source  # Return full HTML content
    except Exception as e:
        print(f"❌ Selenium Error: {e} | URL: {url}")
        return None

# Function to clean extracted names
def clean_name(name):
    exclude_list = ["MarketScreener", "Executive", "Directors", "Chairman", "CEO", "Managing Director",
                    "Non-Executive", "Vice Chairman", "CIMA", "Sri Lanka", "Careers", "Governance","Non-"]
    
    name = name.strip()
    for word in exclude_list:
        name = name.replace(word, "").strip()

    if len(name.split()) < 2 or len(name) < 5:  # Ensure at least 2 words
        return None
    return name

# Function to extract director names
def extract_directors(url):
    html = fetch_page_content(url)
    if not html:
        return []

    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ")

    # Apply NLP
    doc = nlp(text)
    persons = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

    # Extract names near keywords
    relevant_names = set()
    keywords = ["Board of Directors", "Chairman", "CEO", "Director", "Leadership", "Management", "Board Member",
                        "Executive Chairman", "Non-Executive Director","Executive Vice Chairman","Managing Director","Executive Director"]
    for keyword in keywords:
        if keyword in text:
            keyword_pos = text.index(keyword)
            context = text[max(0, keyword_pos - 300): keyword_pos + 300]
            context_doc = nlp(context)
            relevant_names.update([ent.text for ent in context_doc.ents if ent.label_ == "PERSON"])

    return list(filter(None, {clean_name(name) for name in relevant_names}))

# Process each company
failed_results = []

for idx, failed in enumerate(failed_records, 1):
    total = len(failed_records)
    company_name = failed["Company Name"]
    url = failed["URL"]

    print(f"\n🔍 ({idx}/{total}) Processing: {company_name} ({url})")

    director_names = extract_directors(url)

    # Append result to successful_results if directors are found
    if director_names:
        print(f"✅ {company_name} - Extracted Board Members:\n" + "\n".join(director_names))
        successful_results.append({"Company Name": company_name, "URL": url, "Directors": "\n".join(director_names)})
    else:
        print(f"❌ {company_name} - No valid names found.\n")
        failed_results.append({"Company Name": company_name, "URL": url, "Directors": None})

    time.sleep(2)  # Prevent getting blocked

# Add failed results to new DataFrame
df_failed = pd.DataFrame(failed_results)
df_success = pd.DataFrame(successful_results)

# Print the final counts
print(f"\n🆕 Retrying complete.")
print(f"✅ Successfully extracted: {len(successful_results)}")
print(f"❌ Still failed: {len(df_failed)}")




🔍 (1/22) Processing: SAMPATH BANK PLC (https://www.sampath.lk/about-us/leadership)
✅ SAMPATH BANK PLC - Extracted Board Members:
Vajira Kulatilaka
irector
           Read More     
          Rushanka Silva
ng Director
           Read More     
          Rushanka Silva
Managing Dir
Keshini Jayawardena
           
          Independent
Vinod Hirdarama
Vinod Hirdaramani

🔍 (2/22) Processing: BROWNS INVESTMENTS PLC (https://www.brownsinvestments.com/about-us/our-team.html)
✅ BROWNS INVESTMENTS PLC - Extracted Board Members:
Kalsha Amarasinghe
Kapila Jayawardena
Sierra Leone
Ishara Nanayakkara

🔍 (3/22) Processing: CEYLON COLD STORES PLC (https://www.elephanthouse.lk/corporate/our-company/our-team.html)
❌ Selenium Error: Message: unknown error: net::ERR_SSL_VERSION_OR_CIPHER_MISMATCH
  (Session info: chrome=135.0.7049.41)
Stacktrace:
	GetHandleVerifier [0x00007FF7B2081F55+78133]
	GetHandleVerifier [0x00007FF7B2081FB0+78224]
	(No symbol) [0x00007FF7B1E491BA]
	(No symbol) [0x00007FF7B1E45BB6

### Retry again Failures

In [13]:
import time
import warnings
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import spacy
from selenium.common.exceptions import WebDriverException, TimeoutException, InvalidArgumentException

# Load NLP model
nlp = spacy.load("en_core_web_lg")

# Ignore warnings
warnings.filterwarnings("ignore")

successful_results = df_success.to_dict(orient="records")
failed_records = df_failed.to_dict(orient="records")

# Configure Selenium WebDriver (Headless Chrome)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run without opening browser
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")

service = Service()
driver = webdriver.Chrome(service=service, options=chrome_options)

# Function to extract text using Selenium
def fetch_page_content(url):
    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        return driver.page_source  # Return full HTML content
    except (WebDriverException, TimeoutException) as e:
        print(f"❌ Error accessing URL: {url}. Exception: {e}")
        return None
    except InvalidArgumentException as e:
        print(f"❌ Invalid URL: {url}. Exception: {e}")
        return None
    except Exception as e:
        print(f"❌ Unknown error accessing URL: {url}. Exception: {e}")
        return None

# Function to clean extracted names
def clean_name(name):
    exclude_list = ["MarketScreener", "Executive", "Directors", "Chairman", "CEO", "Managing Director",
                    "Non-Executive", "Vice Chairman", "CIMA", "Sri Lanka", "Careers", "Governance","Non-"]
    
    name = name.strip()
    for word in exclude_list:
        name = name.replace(word, "").strip()

    if len(name.split()) < 2 or len(name) < 5:  # Ensure at least 2 words
        return None
    return name

# Function to extract director names
def extract_directors(url):
    html = fetch_page_content(url)
    if not html:
        return []

    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ")

    # Apply NLP
    doc = nlp(text)
    persons = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

    # Extract names near keywords
    relevant_names = set()
    keywords = ["Board of Directors", "Chairman", "CEO", "Director", "Leadership", "Management", "Board Member",
                        "Executive Chairman", "Non-Executive Director","Executive Vice Chairman","Managing Director","Executive Director"]
    for keyword in keywords:
        if keyword in text:
            try:
                keyword_pos = text.index(keyword)
                context = text[max(0, keyword_pos - 300): keyword_pos + 300]
                context_doc = nlp(context)
                relevant_names.update([ent.text for ent in context_doc.ents if ent.label_ == "PERSON"])
            except Exception as e:
                print(f"❌ Error extracting names near keyword '{keyword}': {e}")

    return list(filter(None, {clean_name(name) for name in relevant_names}))

# Process each company
failed_results = []

for idx, failed in enumerate(failed_records, 1):
    total = len(failed_records)
    company_name = failed["Company Name"]
    url = failed["URL"]

    print(f"\n🔍 ({idx}/{total}) Processing: {company_name} ({url})")

    director_names = extract_directors(url)

    # Append result to successful_results if directors are found
    if director_names:
        print(f"✅ {company_name} - Extracted Board Members:\n" + "\n".join(director_names))
        successful_results.append({"Company Name": company_name, "URL": url, "Directors": "\n".join(director_names)})
    else:
        print(f"❌ {company_name} - No valid names found.\n")
        failed_results.append({"Company Name": company_name, "URL": url, "Directors": None})

    time.sleep(2)  # Prevent getting blocked

# Add failed results to new DataFrame
df_failed = pd.DataFrame(failed_results)
df_success = pd.DataFrame(successful_results)

# Print the final counts
print(f"\n🆕 Retrying complete.")
print(f"✅ Successfully extracted: {len(successful_results)}")
print(f"❌ Still failed: {len(df_failed)}")



🔍 (1/16) Processing: CEYLON COLD STORES PLC (https://www.elephanthouse.lk/corporate/our-company/our-team.html)
❌ Error accessing URL: https://www.elephanthouse.lk/corporate/our-company/our-team.html. Exception: Message: unknown error: net::ERR_SSL_VERSION_OR_CIPHER_MISMATCH
  (Session info: chrome=135.0.7049.41)
Stacktrace:
	GetHandleVerifier [0x00007FF7B2081F55+78133]
	GetHandleVerifier [0x00007FF7B2081FB0+78224]
	(No symbol) [0x00007FF7B1E491BA]
	(No symbol) [0x00007FF7B1E45BB6]
	(No symbol) [0x00007FF7B1E36879]
	(No symbol) [0x00007FF7B1E385D8]
	(No symbol) [0x00007FF7B1E36B86]
	(No symbol) [0x00007FF7B1E36606]
	(No symbol) [0x00007FF7B1E362CA]
	(No symbol) [0x00007FF7B1E33F6A]
	(No symbol) [0x00007FF7B1E3474C]
	(No symbol) [0x00007FF7B1E4D08A]
	(No symbol) [0x00007FF7B1EEFF9E]
	(No symbol) [0x00007FF7B1EC715A]
	(No symbol) [0x00007FF7B1EEF21F]
	(No symbol) [0x00007FF7B1EC6F33]
	(No symbol) [0x00007FF7B1E90358]
	(No symbol) [0x00007FF7B1E910C3]
	GetHandleVerifier [0x00007FF7B234BA8

#### html tag parsing for dfcc

In [14]:
# import re
# import requests
# from bs4 import BeautifulSoup
# import spacy

# # Load spaCy model
# nlp = spacy.load("en_core_web_lg")

# def fetch_page_content(url):
#     """Fetch HTML content from a URL with browser headers."""
#     headers = {
#         "User-Agent": (
#             "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
#             "AppleWebKit/537.36 (KHTML, like Gecko) "
#             "Chrome/120.0.0.0 Safari/537.36"
#         )
#     }
#     try:
#         response = requests.get(url, headers=headers, timeout=10)
#         if response.status_code == 200:
#             return response.text
#         else:
#             print(f"Failed to fetch {url}: HTTP {response.status_code}")
#             return None
#     except Exception as e:
#         print(f"Error fetching {url}: {e}")
#         return None

# def clean_name(name):
#     """Remove titles like MR., MRS., etc. from name."""
#     name = re.sub(r"^(MR|MRS|MS|DR)\.?\s+", "", name, flags=re.IGNORECASE).strip()
#     return name

# def extract_directors(url):
#     """Extract potential board member names from a webpage."""
#     html = fetch_page_content(url)
#     if not html:
#         return []

#     soup = BeautifulSoup(html, "html.parser")
#     text = soup.get_text(separator=" ")

#     # Step 1: Extract from <h3> tags using pattern "NAME - ROLE"
#     directors = []
#     for h3 in soup.find_all("h3"):
#         text_content = h3.get_text(separator=" ").strip()
#         match = re.match(r"([A-Z\s\.]+?)\s*-\s*[A-Za-z\s]+", text_content)
#         if match:
#             raw_name = match.group(1)
#             directors.append(clean_name(raw_name))

#     # Step 2: Use spaCy NER to find more PERSON entities
#     doc = nlp(text)
#     persons = [clean_name(ent.text) for ent in doc.ents if ent.label_ == "PERSON"]

#     # Combine and deduplicate
#     all_names = list(set(directors + persons))
#     return all_names

# # 🧪 Example usage
# url = "https://www.dfcc.lk/board-of-directors/"
# directors = extract_directors(url)
# print("\n🎯 Extracted Directors:")
# for name in directors:
#     print("-", name)


#### HTML tag parsing

In [15]:
# import pandas as pd
# import re
# from bs4 import BeautifulSoup
# import spacy

# # Load the spaCy model
# nlp = spacy.load("en_core_web_lg")

# successful_results = df_success.to_dict(orient="records")
# failed_records = df_failed.to_dict(orient="records")

# def clean_name(name):
#     """Clean up name by removing titles like Mr., Mrs., etc."""
#     name = re.sub(r"^MR\.|^MRS\.|^MS\.|^DR\.", "", name).strip()
#     return name

# def extract_directors(url):
#     """Extract directors' names from a given URL."""
#     html = fetch_page_content(url)
#     if not html:
#         return []

#     soup = BeautifulSoup(html, "html.parser")
#     text = soup.get_text(separator=" ")

#     # Extract names from h3 tags, which seem to contain the names and roles
#     directors = []
#     for h3_tag in soup.find_all("h3"):
#         text_content = h3_tag.get_text(separator=" ").strip()
        
#         # Match the pattern of the name and title
#         name_match = re.match(r"([A-Z]+(?:[A-Z\s]+)+)\s+-\s+([A-Za-z\s]+)", text_content)
        
#         if name_match:
#             name = name_match.group(1)  # The name part
#             directors.append(clean_name(name))  # Clean the name and add to the list

#     # Apply NLP for additional extraction (if needed)
#     doc = nlp(text)
#     persons = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]

#     # Combine the director names with the NLP extractions
#     directors = list(set(directors + persons))

#     return directors



# # Process each company
# failed_results = []

# for idx, failed in enumerate(failed_records, 1):
#     total = len(failed_records)
#     company_name = failed["Company Name"]
#     url = failed["URL"]

#     print(f"\n🔍 ({idx}/{total}) Processing: {company_name} ({url})")

#     director_names = extract_directors(url)

#     # Append result to successful_results if directors are found
#     if director_names:
#         print(f"✅ {company_name} - Extracted Board Members:\n" + "\n".join(director_names))
#         successful_results.append({"Company Name": company_name, "URL": url, "Directors": "\n".join(director_names)})
#     else:
#         print(f"❌ {company_name} - No valid names found.\n")
#         failed_results.append({"Company Name": company_name, "URL": url, "Directors": None})

#     time.sleep(2)  # Prevent getting blocked

# # Add failed results to new DataFrame
# df_failed = pd.DataFrame(failed_results)
# df_success = pd.DataFrame(successful_results)

# # Print the final counts
# print(f"\n🆕 Retrying complete.")
# print(f"✅ Successfully extracted: {len(successful_results)}")
# print(f"❌ Still failed: {len(df_failed)}")


In [16]:
import pandas as pd
import requests
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_lg")

def fetch_page_content(url):
    """Fetch the raw text content from a webpage."""
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_names_from_text(text):
    """Extract person names from the text using spaCy NER."""
    doc = nlp(text)
    # Extract entities labeled as PERSON
    persons = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    return persons

successful_results = df_success.to_dict(orient="records")
failed_records = df_failed.to_dict(orient="records")

# Process each company
failed_results = []

for idx, failed in enumerate(failed_records, 1):
    total = len(failed_records)
    company_name = failed["Company Name"]
    url = failed["URL"]

    print(f"\n🔍 ({idx}/{total}) Processing: {company_name} ({url})")

    director_names = extract_directors(url)

    # Append result to successful_results if directors are found
    if director_names:
        print(f"✅ {company_name} - Extracted Board Members:\n" + "\n".join(director_names))
        successful_results.append({"Company Name": company_name, "URL": url, "Directors": "\n".join(director_names)})
    else:
        print(f"❌ {company_name} - No valid names found.\n")
        failed_results.append({"Company Name": company_name, "URL": url, "Directors": None})

    time.sleep(2)  # Prevent getting blocked

# Add failed results to new DataFrame
df_failed = pd.DataFrame(failed_results)
df_success = pd.DataFrame(successful_results)

# Print the final counts
print(f"\n🆕 Retrying complete.")
print(f"✅ Successfully extracted: {len(successful_results)}")
print(f"❌ Still failed: {len(df_failed)}")



🔍 (1/15) Processing: CEYLON COLD STORES PLC (https://www.elephanthouse.lk/corporate/our-company/our-team.html)
Error fetching https://www.elephanthouse.lk/corporate/our-company/our-team.html: HTTPSConnectionPool(host='www.elephanthouse.lk', port=443): Max retries exceeded with url: /corporate/our-company/our-team.html (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))
❌ CEYLON COLD STORES PLC - No valid names found.


🔍 (2/15) Processing: DFCC BANK PLC (https://www.dfcc.lk/board-of-directors/)
❌ DFCC BANK PLC - No valid names found.


🔍 (3/15) Processing: TEEJAY LANKA PLC (https://www.teejay.com/about-teejay/board-of-directors)
Error fetching https://www.teejay.com/about-teejay/board-of-directors: HTTPSConnectionPool(host='www.teejay.com', port=443): Max retries exceeded with url: /about-teejay/board-of-directors (Caused by SSLError(SSLCertVerificationError(1, '[SSL

In [ ]:
import pandas as pd
import requests
import spacy
import time
from bs4 import BeautifulSoup

# Load the spaCy model
nlp = spacy.load("en_core_web_lg")

def fetch_page_content(url):
    """Fetch raw HTML from the URL."""
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to fetch {url}: HTTP {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None

def extract_visible_text(html):
    """Extract clean visible text from HTML."""
    soup = BeautifulSoup(html, 'html.parser')

    # Remove scripts, styles, and non-content elements
    for tag in soup(['script', 'style', 'noscript']):
        tag.decompose()

    text = soup.get_text(separator=' ', strip=True)
    return text

def extract_names_from_text(text):
    """Extract person names using spaCy NER."""
    doc = nlp(text)
    persons = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    return list(set(persons))  # Remove duplicates

def extract_directors(url):
    """Combine all steps: fetch -> clean -> extract names."""
    html = fetch_page_content(url)
    if not html:
        return []

    visible_text = extract_visible_text(html)
    names = extract_names_from_text(visible_text)
    return names

# Convert to dicts
successful_results = df_success.to_dict(orient="records")
failed_records = df_failed.to_dict(orient="records")
failed_results = []

# Process each failed company
for idx, failed in enumerate(failed_records, 1):
    total = len(failed_records)
    company_name = failed["Company Name"]
    url = failed["URL"]

    print(f"\n🔍 ({idx}/{total}) Processing: {company_name} ({url})")

    director_names = extract_directors(url)

    if director_names:
        print(f"✅ {company_name} - Extracted Board Members:\n" + "\n".join(director_names))
        successful_results.append({
            "Company Name": company_name,
            "URL": url,
            "Directors": "\n".join(director_names)
        })
    else:
        print(f"❌ {company_name} - No valid names found.\n")
        failed_results.append({
            "Company Name": company_name,
            "URL": url,
            "Directors": None
        })

    time.sleep(2)  # Pause between requests

# Rebuild dataframes
df_success = pd.DataFrame(successful_results)
df_failed = pd.DataFrame(failed_results)

# Final output
print(f"\n🆕 Retrying complete.")
print(f"✅ Successfully extracted: {len(df_success)}")
print(f"❌ Still failed: {len(df_failed)}")



🔍 (1/15) Processing: CEYLON COLD STORES PLC (https://www.elephanthouse.lk/corporate/our-company/our-team.html)
Error fetching https://www.elephanthouse.lk/corporate/our-company/our-team.html: HTTPSConnectionPool(host='www.elephanthouse.lk', port=443): Max retries exceeded with url: /corporate/our-company/our-team.html (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1000)')))
❌ CEYLON COLD STORES PLC - No valid names found.



In [ ]:
df_success.to_csv("directors_names.csv", index=False)
